# Import library

In [1]:
import os
import os, sys
import glob
import re
import datetime
from datetime import datetime, timedelta 
import math, copy, time
from itertools import cycle
from collections import defaultdict

from parse import parse
import numpy as np
import pandas as pd

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

# import torchvision
# import torchvision.transforms as transforms

from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn import svm


from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import seaborn

%load_ext autoreload
%autoreload 2

In [2]:
pd.set_option("display.max_column", 100)

In [3]:
# Util functions
def size_and_ratio(df, col, dropna=True):
    if pd.__version__ > '1.1':
        sr = df.groupby(col, dropna=dropna).size().sort_values(ascending=False)
    elif dropna:
        sr = df.groupby(col).size().sort_values(ascending=False)
    else:  # dropna == False
        sr = df.fillna('nan').groupby(col).size().sort_values(ascending=False)
    sr_ratio = sr.copy() / sum(sr)
    print("Sum :", sum(sr), sr.shape)
    sr.name = 'size'
    sr_ratio.name = 'ratio'
    return pd.concat([sr, sr_ratio], axis=1)

# Data load

In [4]:
df_train = pd.read_csv("data/df_data_v3_1/df_train_v3_1.csv")
df_val = pd.read_csv("data/df_data_v3_1/df_valid_v3_1.csv")
df_test = pd.read_csv("data/df_data_v3_1/df_test_v3_1.csv")
print(df_train.shape, df_val.shape, df_test.shape)
df_train.head()

(14093, 24) (4647, 24) (4586, 24)


,baby_id,cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday,pcr_result
0,21099,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0.173311,0.007956,0.012987,1,0.0,0.18,3,0
1,21151,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0.179009,0.008614,0.013084,1,0.0,0.17,4,0
2,21024,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.179009,0.008614,0.013084,1,1.0,0.29,4,0
3,21176,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.131500,0.008252,0.000000,0,0.0,0.40,5,0
4,4467,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0.131500,0.008252,0.000000,0,1.0,0.34,5,0


In [5]:
import random
print("Train:", df_train.shape, df_train['baby_id'].nunique())
print("Validaiton:", df_val.shape, df_val['baby_id'].nunique())

Train: (14093, 24) 9641
Validaiton: (4647, 24) 3213


## Remove baby_id and results

In [6]:
exclude_cols = ['baby_id', 'pcr_result']
cols = [x for x in df_train.columns if x not in exclude_cols]

df_train_gt = df_train[['pcr_result']]
df_train_input = df_train[cols]
df_val_gt = df_val[['pcr_result']]
df_val_input = df_val[cols]
df_test_gt = df_test[['pcr_result']]
df_test_input = df_test[cols]


# Make category values

In [7]:
from model.attention_model import CategoricalAttentionModel
from model.utils import get_dict_category_from_dataset
from model.utils import get_category_key, create_cat_and_intensity_from_df

In [8]:
df_trainval = pd.concat([df_train, df_val], axis=0)
print(df_trainval.shape)

dict_category, df_trainval_typed = get_dict_category_from_dataset(df_trainval[cols])
_, df_train_typed = get_dict_category_from_dataset(df_train[cols])
_, df_val_typed = get_dict_category_from_dataset(df_val[cols])
_, df_test_typed = get_dict_category_from_dataset(df_test[cols])
dict_category

(18740, 24)
cough category [1 0] ...
fever category [0 1] ...
sore_throat category [0 1] ...
shortness_of_breath category [0 1] ...
head_ache category [1 0] ...
runny_nose category [0 1] ...
muscle_pain category [0 1] ...
chills category [0 1] ...
loss_of_taste category [0 1] ...
loss_of_smell category [0 1] ...
sputum category [1 0] ...
chest_pain category [0 1] ...
indication_other category [0 1] ...
indication_abroad category [0 1] ...
indication_contact category [1 0] ...
global_confirmed_ratio float [0.1733114  0.17900909 0.13149977 0.14631498 0.15655098 0.17425644
 0.17103502 0.16990724 0.12749546 0.14885249]
confirmed_ratio float [0.00795568 0.00861396 0.00825183 0.00797017 0.01154644 0.01142734
 0.0113018  0.01122937 0.01076423 0.00895517]
sigungu_confirmed_ratio float [0.01298674 0.01308374 0.         0.01324623 0.01342084 0.0135712
 0.01798011 0.01397984 0.01779546 0.0141302 ]
mask category [1 0] ...
gender category [0. 1.] ...
age_ratio float [0.18 0.17 0.29 0.4  0.34 0.41 0

{'cough_1.0': 0,
 'cough_0.0': 1,
 'fever_0.0': 2,
 'fever_1.0': 3,
 'sore_throat_0.0': 4,
 'sore_throat_1.0': 5,
 'shortness_of_breath_0.0': 6,
 'shortness_of_breath_1.0': 7,
 'head_ache_1.0': 8,
 'head_ache_0.0': 9,
 'runny_nose_0.0': 10,
 'runny_nose_1.0': 11,
 'muscle_pain_0.0': 12,
 'muscle_pain_1.0': 13,
 'chills_0.0': 14,
 'chills_1.0': 15,
 'loss_of_taste_0.0': 16,
 'loss_of_taste_1.0': 17,
 'loss_of_smell_0.0': 18,
 'loss_of_smell_1.0': 19,
 'sputum_1.0': 20,
 'sputum_0.0': 21,
 'chest_pain_0.0': 22,
 'chest_pain_1.0': 23,
 'indication_other_0.0': 24,
 'indication_other_1.0': 25,
 'indication_abroad_0.0': 26,
 'indication_abroad_1.0': 27,
 'indication_contact_1.0': 28,
 'indication_contact_0.0': 29,
 'global_confirmed_ratio': 30,
 'confirmed_ratio': 31,
 'sigungu_confirmed_ratio': 32,
 'mask_1.0': 33,
 'mask_0.0': 34,
 'gender_0.0': 35,
 'gender_1.0': 36,
 'age_ratio': 37,
 'weekday_3.0': 38,
 'weekday_4.0': 39,
 'weekday_5.0': 40,
 'weekday_0.0': 41,
 'weekday_1.0': 42,
 'wee

### category 값, intensity 값 생성

### train 데이터 먼저

In [9]:
df_train_category, df_train_intensity = create_cat_and_intensity_from_df(df_train_typed, dict_category)

display(df_train_category.sample(10))
display(df_train_intensity.sample(10))

,cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday
2642,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,35.0,37.0,39.0
12134,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,34.0,36.0,37.0,40.0
12055,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,34.0,36.0,37.0,42.0
13175,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,35.0,37.0,41.0
1050,1.0,3.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,24.0,26.0,28.0,30.0,31.0,32.0,33.0,35.0,37.0,38.0
11360,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,35.0,37.0,40.0
12374,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,36.0,37.0,41.0
7930,0.0,2.0,5.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,36.0,37.0,38.0
12359,1.0,2.0,4.0,6.0,9.0,10.0,12.0,14.0,16.0,18.0,21.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,36.0,37.0,41.0
4008,0.0,2.0,5.0,6.0,8.0,10.0,12.0,14.0,16.0,18.0,20.0,22.0,25.0,26.0,29.0,30.0,31.0,32.0,33.0,35.0,37.0,42.0


,cough,fever,sore_throat,shortness_of_breath,head_ache,runny_nose,muscle_pain,chills,loss_of_taste,loss_of_smell,sputum,chest_pain,indication_other,indication_abroad,indication_contact,global_confirmed_ratio,confirmed_ratio,sigungu_confirmed_ratio,mask,gender,age_ratio,weekday
12403,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.279611,0.173581,0.446118,1.0,1.0,0.29,1.0
2858,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.458350,0.617480,0.150148,1.0,1.0,0.39,1.0
10426,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.117492,0.190635,0.360772,1.0,1.0,0.32,1.0
4434,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.547953,1.000000,0.145338,1.0,1.0,0.45,1.0
10233,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.234417,0.202486,0.000000,1.0,1.0,0.73,1.0
1339,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.362753,0.392090,0.000000,1.0,1.0,0.73,1.0
4498,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.547953,1.000000,0.193543,1.0,1.0,0.38,1.0
4739,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.508594,0.655087,0.210881,1.0,1.0,0.41,1.0
10449,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.117492,0.190635,0.000000,1.0,1.0,0.38,1.0
6963,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.362548,0.559093,0.000000,1.0,1.0,0.36,1.0


예시로 sore_throat id값 확인해봄

In [10]:
size_and_ratio(df_train_category, 'sore_throat')

Sum : 14093 (2,)


,size,ratio
sore_throat,,
5.0,7723,0.548003
4.0,6370,0.451997


### df_val에서 df_cat값과 df_intensity 값 생성

In [11]:
df_val_category, df_val_intensity = create_cat_and_intensity_from_df(df_val_typed, dict_category)


### df_test에서도 마찬가지로 생성

In [12]:
df_test_category, df_test_intensity = create_cat_and_intensity_from_df(df_test_typed, dict_category)


In [13]:
train_np_cat = df_train_category.values
train_np_ins = df_train_intensity.values
train_np_gt = df_train_gt[['pcr_result']].values

val_np_cat = df_val_category.values
val_np_ins = df_val_intensity.values
val_np_gt = df_val_gt[['pcr_result']].values

test_np_cat = df_test_category.values
test_np_ins = df_test_intensity.values
test_np_gt = df_test_gt[['pcr_result']].values

train_np_gt, val_np_gt, test_np_gt

(array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [0]]),
 array([[0],
        [0],
        [0],
        ...,
        [0],
        [1],
        [0]]),
 array([[0],
        [0],
        [0],
        ...,
        [1],
        [0],
        [1]]))

In [14]:
train_input = [train_np_cat.copy(), train_np_ins.copy()]
train_gt = train_np_gt.copy()

valid_input = [val_np_cat.copy(), val_np_ins.copy()]
valid_gt = val_np_gt.copy()

test_input = [test_np_cat.copy(), test_np_ins.copy()]
test_gt = test_np_gt.copy()


In [15]:
print(train_input[0].shape, valid_input[0].shape, test_input[0].shape)
print(train_gt.shape, valid_gt.shape, test_gt.shape)

(14093, 22) (4647, 22) (4586, 22)
(14093, 1) (4647, 1) (4586, 1)


In [16]:
print("# of train data positive: ", train_gt.sum())
print("# of train data negative: ", train_gt.shape[0] - train_gt.sum())
print("ratio of P/N: ", train_gt.sum()/(train_gt.shape[0] - train_gt.sum()))
print("")

print("# of valid data positive: ", valid_gt.sum())
print("# of valid data negative: ", valid_gt.shape[0] - valid_gt.sum())
print("ratio of P/N: ", valid_gt.sum()/(valid_gt.shape[0] - valid_gt.sum()))
print("")

print("# of test data positive: ", test_gt.sum())
print("# of test data negative: ", test_gt.shape[0] - test_gt.sum())
print("ratio of P/N: ", test_gt.sum()/(test_gt.shape[0] - test_gt.sum()))

# of train data positive:  10063
# of train data negative:  4030
ratio of P/N:  2.4970223325062033

# of valid data positive:  3415
# of valid data negative:  1232
ratio of P/N:  2.7719155844155843

# of test data positive:  3332
# of test data negative:  1254
ratio of P/N:  2.657097288676236


### Check dict_category

In [17]:
dict_category

{'cough_1.0': 0,
 'cough_0.0': 1,
 'fever_0.0': 2,
 'fever_1.0': 3,
 'sore_throat_0.0': 4,
 'sore_throat_1.0': 5,
 'shortness_of_breath_0.0': 6,
 'shortness_of_breath_1.0': 7,
 'head_ache_1.0': 8,
 'head_ache_0.0': 9,
 'runny_nose_0.0': 10,
 'runny_nose_1.0': 11,
 'muscle_pain_0.0': 12,
 'muscle_pain_1.0': 13,
 'chills_0.0': 14,
 'chills_1.0': 15,
 'loss_of_taste_0.0': 16,
 'loss_of_taste_1.0': 17,
 'loss_of_smell_0.0': 18,
 'loss_of_smell_1.0': 19,
 'sputum_1.0': 20,
 'sputum_0.0': 21,
 'chest_pain_0.0': 22,
 'chest_pain_1.0': 23,
 'indication_other_0.0': 24,
 'indication_other_1.0': 25,
 'indication_abroad_0.0': 26,
 'indication_abroad_1.0': 27,
 'indication_contact_1.0': 28,
 'indication_contact_0.0': 29,
 'global_confirmed_ratio': 30,
 'confirmed_ratio': 31,
 'sigungu_confirmed_ratio': 32,
 'mask_1.0': 33,
 'mask_0.0': 34,
 'gender_0.0': 35,
 'gender_1.0': 36,
 'age_ratio': 37,
 'weekday_3.0': 38,
 'weekday_4.0': 39,
 'weekday_5.0': 40,
 'weekday_0.0': 41,
 'weekday_1.0': 42,
 'wee

## Let's Train imported class (simple attention module without inesity encoding)

In [18]:
from model.attention_model import CategoricalAtentionMultiplyIntensityModel, CategoricalAttentionAddIntensityModel
from model.simple_attention import CategoricalAttentionSimpleModel
from model.base_dataloader import IsraelDataLoader, IsraelDatasetWithIntensity
from model.utils import run_validation

In [19]:
train_input

[array([[ 0.,  2.,  4., ..., 35., 37., 38.],
        [ 1.,  2.,  5., ..., 35., 37., 39.],
        [ 1.,  2.,  5., ..., 36., 37., 39.],
        ...,
        [ 0.,  2.,  5., ..., 36., 37., 41.],
        [ 1.,  2.,  4., ..., 36., 37., 41.],
        [ 1.,  2.,  4., ..., 36., 37., 41.]]),
 array([[1.  , 1.  , 1.  , ..., 1.  , 0.18, 1.  ],
        [1.  , 1.  , 1.  , ..., 1.  , 0.17, 1.  ],
        [1.  , 1.  , 1.  , ..., 1.  , 0.29, 1.  ],
        ...,
        [1.  , 1.  , 1.  , ..., 1.  , 0.31, 1.  ],
        [1.  , 1.  , 1.  , ..., 1.  , 0.29, 1.  ],
        [1.  , 1.  , 1.  , ..., 1.  , 0.46, 1.  ]])]

In [20]:
print("cat value:", train_input[0].shape)
print("intensity value:", train_input[1].shape)
print("Ground truth value:", train_gt.shape)

cat value: (14093, 22)
intensity value: (14093, 22)
Ground truth value: (14093, 1)


# Dataset, data_loader

In [21]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [22]:
# creat dataset

train_dataset = IsraelDatasetWithIntensity(
    train_np_cat,
    train_np_ins,
    train_np_gt,
    dict_category
)

val_dataset = IsraelDatasetWithIntensity(
    val_np_cat,
    val_np_ins,
    val_np_gt,
    dict_category
)

test_dataset = IsraelDatasetWithIntensity(
    test_np_cat, 
    test_np_ins,
    test_np_gt,
    dict_category
)

## Transformer encoder
일단 3층짜리로 한번 돌려보자

In [23]:
from model.attention_model import CategoricalTransformerEncoderMultiplyIntensity

In [38]:
def save_model(config_name, model, epoch):
    save_path = f"./train_data/{model.__class__.__name__}_{config_name}_{epoch:04d}.pth"
    print("save_path", save_path)
    torch.save(model.state_dict(), save_path)
    print("Success to save to : ", save_path)
    return model

batch_size = 32
epochs = 300

train_loader = IsraelDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = IsraelDataLoader(val_dataset, batch_size=batch_size, shuffle=False)

len_seq = train_input[0].shape[1]
print("len_seq:", len_seq)
d_model = 64
n_head = 8
n_layer = 3
lr = 1e-5

encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_head, dropout=0.1)
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layer)
model = CategoricalTransformerEncoderMultiplyIntensity(
    transformer_encoder=transformer_encoder,
    dict_category=dict_category,
    len_seq=len_seq,
)

model.eval()

print("d_model:", model.d_model, "n_head:", model.n_head, 'len_seq:', model.len_seq)
loss_func = nn.BCELoss(reduction='mean')
optimizer = optim.RAdam(model.parameters(), lr=lr)
config_name = f"big_layer{n_layer}_dmodel{d_model}_nhead{n_head}_lr{lr}"

train_log = dict()
val_log = dict()

val_metrics = {
    "loss": loss_func,
    'auc': lambda pred, gt: roc_auc_score(gt.numpy(), pred.numpy())
}


for epoch in range(epochs):
    # **************** Train ****************
    train_result_info = CategoricalTransformerEncoderMultiplyIntensity.train_on_epoch(epoch, train_loader, model, loss_func, optimizer, verbose=True)
    train_log[f'epoch_train_{epoch}'] = train_result_info
    logger.debug(f"epoch {epoch} train finished. train_result_info:{train_result_info}")
    
    # ************** validation *********************
    val_result = CategoricalTransformerEncoderMultiplyIntensity.validation_on_epoch(epoch, model, val_loader, val_metrics)
    val_log[f'epoch_val_{epoch}'] = val_result
    logger.info(f"epoch {epoch} val result: {val_result}")

    if not val_log:
        best_model = save_model(config_name, model.copy(), epoch)
    elif val_result['auc'] >= max(val_log.values(), key=lambda x : x['auc'])['auc']:
        best_model = save_model(config_name, model.copy(), epoch)

    

len_seq: 22
d_model: 64 n_head: 8 len_seq: 22


INFO:model.base_model:Epoch Step: 0 Train Loss: 0.6904242038726807 elapsed: 21.818186283111572
INFO:model.base_model:cur_epoch:0, validation elapsed_time:1.613265037536621
INFO:__main__:epoch 0 val result: {'loss': tensor(0.6451), 'auc': 0.7917642752562226}


save_path ./train_data/CategoricalTransformerEncoderMultiplyIntensity_big_layer3_dmodel64_nhead8_lr1e-05_0000.pth
Success to save to :  ./train_data/CategoricalTransformerEncoderMultiplyIntensity_big_layer3_dmodel64_nhead8_lr1e-05_0000.pth


INFO:model.base_model:Epoch Step: 1 Train Loss: 0.5643001794815063 elapsed: 23.116697072982788
INFO:model.base_model:cur_epoch:1, validation elapsed_time:1.9859809875488281
INFO:__main__:epoch 1 val result: {'loss': tensor(0.5157), 'auc': 0.794168322526668}


save_path ./train_data/CategoricalTransformerEncoderMultiplyIntensity_big_layer3_dmodel64_nhead8_lr1e-05_0001.pth
Success to save to :  ./train_data/CategoricalTransformerEncoderMultiplyIntensity_big_layer3_dmodel64_nhead8_lr1e-05_0001.pth


...

INFO:model.base_model:Epoch Step: 232 Train Loss: 0.46371015906333923 elapsed: 33.96895408630371
INFO:model.base_model:cur_epoch:232, validation elapsed_time:1.9023520946502686
INFO:__main__:epoch 232 val result: {'loss': tensor(0.5504), 'auc': 0.8179058679241695}


save_path ./train_data/CategoricalTransformerEncoderMultiplyIntensity_big_layer3_dmodel64_nhead8_lr1e-05_0232.pth
Success to save to :  ./train_data/CategoricalTransformerEncoderMultiplyIntensity_big_layer3_dmodel64_nhead8_lr1e-05_0232.pth


INFO:model.base_model:Epoch Step: 233 Train Loss: 0.4720337688922882 elapsed: 32.07291603088379
INFO:model.base_model:cur_epoch:233, validation elapsed_time:1.9777381420135498
INFO:__main__:epoch 233 val result: {'loss': tensor(0.4468), 'auc': 0.8178219657355821}
INFO:model.base_model:Epoch Step: 234 Train Loss: 0.48448145389556885 elapsed: 31.89808487892151
INFO:model.base_model:cur_epoch:234, validation elapsed_time:1.985658884048462
INFO:__main__:epoch 234 val result: {'loss': tensor(0.4676), 'auc': 0.8174086345572436}
INFO:model.base_model:Epoch Step: 235 Train Loss: 0.5042496919631958 elapsed: 32.435646772384644
INFO:model.base_model:cur_epoch:235, validation elapsed_time:1.968446969985962
INFO:__main__:epoch 235 val result: {'loss': tensor(0.4558), 'auc': 0.815942723089502}
INFO:model.base_model:Epoch Step: 236 Train Loss: 0.5145543217658997 elapsed: 33.24068093299866
INFO:model.base_model:cur_epoch:236, validation elapsed_time:1.93123197555542
INFO:__main__:epoch 236 val result:

INFO:model.base_model:cur_epoch:264, validation elapsed_time:1.8749566078186035
INFO:__main__:epoch 264 val result: {'loss': tensor(0.4879), 'auc': 0.7908244756707421}
INFO:model.base_model:Epoch Step: 265 Train Loss: 0.5542016625404358 elapsed: 31.467709064483643
INFO:model.base_model:cur_epoch:265, validation elapsed_time:1.9063029289245605
INFO:__main__:epoch 265 val result: {'loss': tensor(0.5890), 'auc': 0.7906029548782112}
INFO:model.base_model:Epoch Step: 266 Train Loss: 0.5160369873046875 elapsed: 30.47427988052368
INFO:model.base_model:cur_epoch:266, validation elapsed_time:1.944577932357788
INFO:__main__:epoch 266 val result: {'loss': tensor(0.5459), 'auc': 0.7901508813295051}
INFO:model.base_model:Epoch Step: 267 Train Loss: 0.5544703602790833 elapsed: 30.42290425300598
INFO:model.base_model:cur_epoch:267, validation elapsed_time:1.817868709564209
INFO:__main__:epoch 267 val result: {'loss': tensor(0.4854), 'auc': 0.789407645794908}
INFO:model.base_model:Epoch Step: 268 Trai

INFO:__main__:epoch 295 val result: {'loss': tensor(0.5157), 'auc': 0.798947182027343}
INFO:model.base_model:Epoch Step: 296 Train Loss: 0.5185495615005493 elapsed: 33.3410370349884
INFO:model.base_model:cur_epoch:296, validation elapsed_time:1.9925000667572021
INFO:__main__:epoch 296 val result: {'loss': tensor(0.4778), 'auc': 0.799133882223194}
INFO:model.base_model:Epoch Step: 297 Train Loss: 0.5024392008781433 elapsed: 34.05741596221924
INFO:model.base_model:cur_epoch:297, validation elapsed_time:1.9920480251312256
INFO:__main__:epoch 297 val result: {'loss': tensor(0.5158), 'auc': 0.7993851134224487}
INFO:model.base_model:Epoch Step: 298 Train Loss: 0.49499598145484924 elapsed: 35.02268695831299
INFO:model.base_model:cur_epoch:298, validation elapsed_time:2.138288974761963
INFO:__main__:epoch 298 val result: {'loss': tensor(0.4909), 'auc': 0.799206019090719}
INFO:model.base_model:Epoch Step: 299 Train Loss: 0.5184526443481445 elapsed: 33.16764497756958
INFO:model.base_model:cur_ep

In [39]:
## Check the best model's validation performance
display(val_log)
print("Best validation performance:", max(val_log.values(), key=lambda x : x['auc']))

## Check the best model's test performance
test_loader = IsraelDataLoader(test_dataset, batch_size=64, shuffle=False)

best_model.eval()
test_result = best_model.validation_on_epoch(epoch, best_model, test_loader, val_metrics)
test_result

{'epoch_val_0': {'loss': tensor(0.6451), 'auc': 0.7917642752562226},
 'epoch_val_1': {'loss': tensor(0.5157), 'auc': 0.794168322526668},
 'epoch_val_2': {'loss': tensor(0.4797), 'auc': 0.7912326966591243},
 'epoch_val_3': {'loss': tensor(0.5523), 'auc': 0.7925507691430093},
 'epoch_val_4': {'loss': tensor(0.6213), 'auc': 0.8002505181494932},
 'epoch_val_5': {'loss': tensor(0.4613), 'auc': 0.8022160398166988},
 'epoch_val_6': {'loss': tensor(0.4818), 'auc': 0.7967079680933999},
 'epoch_val_7': {'loss': tensor(0.6454), 'auc': 0.7973350953585214},
 'epoch_val_8': {'loss': tensor(0.5814), 'auc': 0.7989383877469529},
 'epoch_val_9': {'loss': tensor(0.5852), 'auc': 0.7993666929702802},
 'epoch_val_10': {'loss': tensor(0.4821), 'auc': 0.7937347882717576},
 'epoch_val_11': {'loss': tensor(0.4925), 'auc': 0.7831509906638018},
 'epoch_val_12': {'loss': tensor(0.5098), 'auc': 0.7729354832575916},
 'epoch_val_13': {'loss': tensor(0.4720), 'auc': 0.7977689861383127},
 'epoch_val_14': {'loss': tenso

Best validation performance: {'loss': tensor(0.5504), 'auc': 0.8179058679241695}


INFO:model.base_model:cur_epoch:299, validation elapsed_time:1.6951000690460205


{'loss': tensor(0.5510), 'auc': 0.822630966262103}